In [1]:
import pandas as pd
from Bio import SeqIO
import numpy as np
import pickle
from Bio.Seq import Seq
import matplotlib.pyplot as plt
import seaborn as sns
stops = set(["TAA","TAG","TGA"])
starts = set(["ATG", "TTG", "GTG","CTG"])

In [4]:
df = pd.read_csv("TE2.tsv", sep="\t",)
colnames = df.columns[1:-1]
tissue2expr = {}
for i in range(len(df)):
    row = df.iloc[i]
    for name in colnames:
        if name not in tissue2expr:
            tissue2expr[name] = []
        if row[name] != row[name]: continue
        tissue2expr[name].append(row[name])
tissue2thr = {}
for k,v in tissue2expr.items():
    tissue2thr[k] = (np.quantile(v, 0.5), np.quantile(v, 0.5))

tissue2id = {}
for i in range(len(df)):
    row = df.iloc[i]
    enst = row["ENSG"]
    for name in colnames:
        highthr,lowthr = tissue2thr[name]
        if row[name] != row[name]: continue
        expr = row[name]
        if name not in tissue2id:
            tissue2id[name] = ([] , [])
        if expr > highthr:
            tissue2id[name][0].append(enst)
        elif expr < lowthr:
            tissue2id[name][1].append(enst)
id2cnt = {}
for k,v in tissue2id.items():
    highs,lows = v
    for high in highs:
        if high not in id2cnt:
            id2cnt[high] = [0,0]
        id2cnt[high][0] += 1
    for low in lows:
        if low not in id2cnt:
            id2cnt[low] = [0,0]
        id2cnt[low][1] += 1
idhighcnt = []; idlowcnt = []
for k,v in id2cnt.items():
    idhighcnt.append(v[0])
    idlowcnt.append(v[1])
highensg = []
lowensg = []

for k,v in id2cnt.items():
    if (v[0] > 3 and v[1] > 3) and k.startswith("ENSG"):
        # print(k,v)
        pass
    if not k.startswith("ENSG"): continue
    if v[0] > 2 and v[1] < 2:
        highensg.append(k)
    elif v[1] > 2 and v[0] < 2:
        lowensg.append(k)
print(len(highensg), len(lowensg))
SEED = 42
np.random.seed(SEED)
lowensg = np.random.choice(lowensg, len(highensg), replace=False)
print(len(highensg), len(lowensg))


4818 5632
4818 4818


In [6]:
phastEI = {}
with open("phastConsElements30way.txt") as f:
    for q in f:
        line = q.split("\t")
        chr = line[1]
        start = int(line[2]); end = int(line[3]); score = int(line[5])
        if chr not in phastEI:
            phastEI[chr] = [[],[],[]]
        phastEI[chr][0].append(start); phastEI[chr][1].append(end); phastEI[chr][2].append(score)

In [8]:
# for tisn in tissue2id.keys():
if True:
    # highensg = tissue2id[tisn][0]
    # lowensg = tissue2id[tisn][1]
    for idx,ensg_list in enumerate([highensg, lowensg]):
        ensg2strand = {}
        ensg25utr_region = {}
        with open("five_prime_UTR.MANE.GRCh38.v1.3.ensembl_genomic.gff") as f:
            for q in f:
                line = q.rstrip().split("\t")
                ensg = line[8].split("gene_id=")[1].split(".")[0]
                start = int(line[3])
                end = int(line[4])
                chr = line[0]
                strand = line[6]
                ensg2strand[ensg] = strand
                if ensg not in ensg_list: continue
                if ensg not in ensg25utr_region:
                    ensg25utr_region[ensg] = [chr]
                ensg25utr_region[ensg].append((start,end))
        ensg2phast = {}
        for ensg, v in ensg25utr_region.items():
            chr = v[0]
            
            ensg2phast[ensg] = {'chr':v[0], 'region':v[1:], 'phast':[]}
            phast = []
            tmp_region = sorted(v[1:], key=lambda x:x[0])
            ensg2phast[ensg]['region'] = tmp_region
            for start,end in tmp_region:
                star_idx = np.searchsorted(phastEI[chr][0],start)
                end_idx = np.searchsorted(phastEI[chr][1],end)
                if phastEI[chr][1][star_idx-1] > start:
                    if phastEI[chr][1][star_idx-1] < end:
                        phast.append((start,phastEI[chr][1][star_idx-1], phastEI[chr][2][star_idx-1]))
                    else:
                        phast.append((start,end, phastEI[chr][2][star_idx-1]))
                for i in range(star_idx,end_idx):
                    phast.append((phastEI[chr][0][i],phastEI[chr][1][i],phastEI[chr][2][i]))
                if end_idx >= star_idx and phastEI[chr][0][end_idx] < end:
                    phast.append((phastEI[chr][0][end_idx],end, phastEI[chr][2][end_idx]))
            ensg2phast[ensg]['phast'] = phast
        # filename = f"comparison/{tisn}_class5.bed" if idx==0 else f"comparison/{tisn}_class1.bed"
        filename = f"all_class5.bed" if idx==0 else f"all_class1.bed"
        with open(filename, 'w') as w:
            for ensg, v in ensg2phast.items():
                chr = v['chr']
                region = v['region']
                phast = v['phast']
                strand = ensg2strand[ensg]
                for start,end,score in phast:
                    # if score < 500: continue
                    # if start > end - 100: continue
                    w.write(f"{chr}\t{start}\t{end}\t{ensg}_{start}_{end}_{score}\t0\t{strand}\n")
        

In [ ]:
!bedtools getfasta -fi /data/RefData/hg38.fa -bed all_class5.bed -s -name > all_class5.raw.fa  
!bedtools getfasta -fi /data/RefData/hg38.fa -bed all_class1.bed -s -name > all_class1.raw.fa  

In [10]:
records = SeqIO.parse("all_class1.raw.fa", "fasta")
with open("all_class1.fa", 'w') as w:
    for record in records:
        id  = str(record.id).split("::")[0]
        w.write(f">{id}\n{str(record.seq).upper()}\n")
records = SeqIO.parse("all_class5.raw.fa", "fasta")
with open("all_class5.fa", 'w') as w:
    for record in records:
        id  = str(record.id).split("::")[0]
        w.write(f">{id}\n{str(record.seq).upper()}\n")

In [ ]:
!cd-hit -i all_class5.fa -o all_class5.filtered.fa -T 8  
!cd-hit -i all_class1.fa -o all_class1.filtered.fa -T 8 

!makeblastdb -in all_class5.filtered.fa -dbtype nucl -out DB/ac5
!makeblastdb -in all_class1.filtered.fa -dbtype nucl -out DB/ac1